In [1]:
import pandas as pd

import wandb

# Sweep

In [2]:
api = wandb.Api()

In [3]:
from pydantic import BaseModel

In [4]:
class SweepData(BaseModel):
    sweep_id: str
    train_accuracy: str
    validation_accuracy: str
    test_accuracy: str

In [13]:
SWEEPS: dict[str, SweepData] = {
    "CNN-NO-PREP": SweepData(
        sweep_id="silvan-wiedmer-fhgr/VisionTransformer/k1h9h3lh",
        train_accuracy="epoch/accuracy",
        validation_accuracy="epoch/val_accuracy",
        test_accuracy="test_acc",
    ),
    "INCEPTION-PREP": SweepData(
        sweep_id="silvan-wiedmer-fhgr/VisionTransformer/5ghtealo",
        train_accuracy="epoch/accuracy",
        validation_accuracy="epoch/val_accuracy",
        test_accuracy="test_acc",
    ),
    "VISION-TRANSFORMER-PREP": SweepData(
        sweep_id="silvan-wiedmer-fhgr/VisionTransformer/fpspjbrh",
        train_accuracy="train_accuracy",
        validation_accuracy="val_accuracy",
        test_accuracy="test_acc",
    ),
    "VISION-TRANSFORMER-LITE-PREP": SweepData(
        sweep_id="silvan-wiedmer-fhgr/VisionTransformer/kdbpf14a",
        train_accuracy="train_accuracy",
        validation_accuracy="val_accuracy",
        test_accuracy="test_acc",
    ),
}

In [14]:
model_metrics = {
    "Model": [],
    "Training Accuracy": [],
    "Validation Accuracy": [],
    "Test Accuracy": [],
    "F1 Score": [],
}

In [15]:
for sweep_name, sweep_data in SWEEPS.items():
    sweep = api.sweep(sweep_data.sweep_id)

    runs = sorted(sweep.runs, key=lambda run: run.summary.get(sweep_data.test_accuracy, 0), reverse=True)
    train_acc = runs[0].summary.get(sweep_data.train_accuracy, 0)
    val_acc = runs[0].summary.get(sweep_data.validation_accuracy, 0)
    test_acc = runs[0].summary.get(sweep_data.test_accuracy, 0)
    model_metrics["Model"].append(sweep_name)
    model_metrics["Training Accuracy"].append(train_acc)
    model_metrics["Validation Accuracy"].append(val_acc)
    model_metrics["Test Accuracy"].append(test_acc)
    model_metrics["F1 Score"].append(0)
    print(f"Best run for {sweep_name} {runs[0].name} with {val_acc} validation accuracy")


Best run for CNN-NO-PREP lr_0.01_bs_16_do_0.20 with 0.6919642686843872 validation accuracy
Best run for INCEPTION-PREP lr_0.0001_bs_16_do_0.40 with 0.7863247990608215 validation accuracy
Best run for VISION-TRANSFORMER-PREP lr_0.001_bs_16_do_0.40 with 78.89908256880734 validation accuracy
Best run for VISION-TRANSFORMER-LITE-PREP lr_0.001_bs_64_do_0.20 with 70.64220183486239 validation accuracy


In [16]:
df_system_metrics = runs[0].history(stream="system")
df_system_metrics.head()

,_wandb,system.memory_percent,_runtime,system.proc.memory.rssMB,system.proc.memory.availableMB,system.proc.memory.percent,_timestamp,system.network.sent,system.network.recv,system.disk./.usageGB,...,system.cpu.58.cpu_percent,system.cpu.57.cpu_percent,system.cpu.55.cpu_percent,system.cpu.8.cpu_percent,system.cpu.59.cpu_percent,system.cpu.38.cpu_percent,system.cpu.18.cpu_percent,system.cpu.17.cpu_percent,system.cpu.14.cpu_percent,system.cpu.30.cpu_percent
0,True,2.969296,10.353970,1455.101562,496832.835938,0.282185,1.730408e+09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,NaN,10.354220,NaN,NaN,NaN,1.730408e+09,432701.0,402847.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,NaN,10.354928,NaN,NaN,NaN,1.730408e+09,NaN,NaN,0.000015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,NaN,10.357838,NaN,NaN,NaN,1.730408e+09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,NaN,10.358836,NaN,NaN,NaN,1.730408e+09,NaN,NaN,NaN,...,0.549451,0.274474,71.590909,13.09329,0.091491,72.014925,72.462406,72.567192,0.182648,0.365965


In [17]:
df_system_metrics.columns

Index(['_wandb', 'system.memory_percent', '_runtime',
       'system.proc.memory.rssMB', 'system.proc.memory.availableMB',
       'system.proc.memory.percent', '_timestamp', 'system.network.sent',
       'system.network.recv', 'system.disk./.usageGB', 'system.disk.out',
       'system.disk./.usagePercent', 'system.disk.in',
       'system.gpu.0.powerPercent', 'system.gpu.0.smClock',
       'system.gpu.0.powerWatts', 'system.gpu.0.temp', 'system.gpu.0.memory',
       'system.gpu.0.gpu', 'system.gpu.0.correctedMemoryErrors',
       'system.gpu.0.memoryAllocatedBytes', 'system.gpu.0.pcieLinkGen',
       'system.gpu.0.uncorrectedMemoryErrors', 'system.gpu.0.memoryAllocated',
       'system.gpu.0.memoryClock', 'system.gpu.0.enforcedPowerLimitWatts',
       'system.cpu.60.cpu_percent', 'system.cpu.52.cpu_percent',
       'system.cpu.42.cpu_percent', 'system.cpu.22.cpu_percent',
       'system.cpu.9.cpu_percent', 'system.cpu.37.cpu_percent',
       'system.cpu.1.cpu_percent', 'system.cpu.49.c

In [18]:
df_system_metrics["system.cpu"]

0          NaN
1          NaN
2          NaN
3          NaN
4     4.190797
5          NaN
6          NaN
7          NaN
8     4.192393
9          NaN
10         NaN
11         NaN
12         NaN
13         NaN
14    4.200728
15         NaN
16         NaN
17         NaN
18         NaN
19    4.209410
20         NaN
21         NaN
22         NaN
23         NaN
24    4.224742
25         NaN
26         NaN
27         NaN
28    4.231813
29         NaN
30         NaN
31         NaN
32         NaN
33    4.240686
34         NaN
35         NaN
36         NaN
37         NaN
38    4.246488
39         NaN
40         NaN
41         NaN
42         NaN
43    4.249587
Name: system.cpu, dtype: float64

In [19]:
df_metrics = pd.DataFrame(model_metrics)
df_metrics.head()

,Model,Training Accuracy,Validation Accuracy,Test Accuracy,F1 Score
0,CNN-NO-PREP,0.696175,0.691964,0.743151,0
1,INCEPTION-PREP,0.924546,0.786325,0.842466,0
2,VISION-TRANSFORMER-PREP,81.256133,78.899083,84.018265,0
3,VISION-TRANSFORMER-LITE-PREP,72.914622,70.642202,82.191781,0


In [20]:
df_metrics.to_latex(
    index=False,
    formatters={"name": str.upper},
    float_format="{:.4f}".format,
    buf="model_metrics.tex"
)